In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

34123


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [6]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
add_publication = (
    "INSERT INTO publication "
    "(id, title, doi, author, year) "
    "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
)
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
add_histone_has_publication = (
    "INSERT INTO histone_has_publication "
    "(histone_id, publication_id) "
    "VALUES (%s, %s)"
)

# To Do

1. Add to description of cenH3_(Plants):

    ```
    While most diploid eukaryotes and flowering plants maintain only one CENH3 gene copy (even after whole-genome duplications), some diploid species like Arabidopsis lyrata, barley, rye, pea, and related legumes retain two functional CENH3 homologs [ishii_unequal_2020].
    ```

2. Add node cenH3_(Vigna_unguiculata) before children nodes cenH3.1_(Vigna_unguiculata) and cenH3.2_(Vigna_unguiculata).
3. Add node cenH3_(Wheat) after cenH3_(Plants).
4. Add two children nodes for cenH3_(Triticeae): cenH3.1_(Triticeae) and cenH3.1_(Triticeae)
5. Add descriptions and publication for new nodes [yuan_characterization_2015]
6. Add sequences for cenH3.1_(Triticeae) and cenH3.1_(Triticeae)


*See more in DeepSeek "Отличия αCENH3 и βCENH3 в пшенице"*

Вот структурированная выгрузка ключевых выводов из нашего обсуждения статьи **Yuan et al. (2015)** о генах *αCENH3* и *βCENH3* у пшеницы и родственных видов:

---

### **1. Основные открытия статьи [yuan_characterization_2015]**  
- **Два гена *CENH3*** (*α* и *β*) обнаружены у:  
  - Всех изученных видов *Triticum* (диплоиды, тетраплоиды, гексаплоиды).  
  - Диплоидных предков (*Aegilops speltoides* (SS), *Ae. tauschii* (DD)).  
  - Ячменя (*Hordeum vulgare*).  
- **Функциональные различия**:  
  - *αCENH3* критичен для митоза и развития растений. Подавление экспрессии *αCENH3* приводит к сильной карликовости и ослаблению корневой системы, что указывает на его критическую роль в развитии растения.
  - *βCENH3* важен для репродукции и адаптации полиплоидов. Подавление *βCENH3* вызывает менее выраженные фенотипические изменения, но снижает репродуктивную способность, что предполагает его специализированную роль в репродуктивном развитии.

**Цитата**:  
> *"Plants inoculated with BSMV:αCENH3 were extremely dwarfed [...] while BSMV:βCENH3-treated plants produced very few seeds."*  

---

### **2. Таксономические уточнения**  
- **Не все злаки имеют два гена**:  
  - Один ген: рис (*Oryza*), кукуруза (*Zea*), сорго (*Sorghum*).  
  - Два гена: пшеница (*Triticum*), ячмень (*Hordeum*), *Aegilops*.  
- **Aegilops ≠ пшеница**:  
  - *Ae. speltoides* (SS) — дикий предок B-генома пшеницы, но относится к другому роду.  

**Цитата**:  
> *"Tetraploid wheat evolved through hybridization between Triticum (AA) and Aegilops (SS)."*  

---

### **3. Методология**  
- **Экспериментальное открытие *βCENH3***:  
  - ПЦР с праймерами P1/P2 на консервативном участке CATD.  
  - Не биоинформатика (BLAST выявил только *αCENH3*).  
- **Подтверждение**:  
  - Секвенирование, FISH, VIGS, филогенетический анализ.  

**Цитата**:  
> *"Using primers P1 and P2, we identified a second CENH3, named βCENH3."*  

---

### **4. Ограничения и открытые вопросы**  
- **Не все *Triticinae* изучены**:  
  - Нет данных по родам *Ambylopyrum*, *Henrardia* и др.  
- **Причины дупликации**:  
  - Предполагается общий предок *Pooideae* (~13 млн лет назад), но механизмы не ясны.  

---

### **5. Ключевые цитаты**  
1. О роли *βCENH3*:  
   > *"βCENH3 is under positive selection in cultivated tetraploids, suggesting adaptive evolution."*  
2. О локализации:  
   > *"αCENH3 is present at all mitotic stages, while βCENH3 is detected mainly in interphase."*  


# Close connections

In [35]:
cursor.close()
conn.close()
tunnel.stop()